# TP1 Projeto de Bloco: Classificação de Doenças Cardíacas com KNN

Este projeto propõe uma experiência prática em Inteligência Artificial (IA) e Machine Learning (ML), permitindo a aplicação de conceitos e técnicas adquiridos em disciplinas regulares num contexto de dados reais.

## 1. Carregamento de Dados



In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

In [ ]:
try:
    df = pd.read_csv('heart.csv')
except FileNotFoundError:
    print("Erro: O arquivo 'heart.csv' não foi encontrado. Verifique o caminho do arquivo.")


print("Primeiras 5 linhas do dataset:")
print(df.head())
print("\nInformações do dataset:")
df.info()
print("\nDescrição estatística do dataset (numérico):")
print(df.describe())
print("\nDescrição estatística do dataset (categórico):")
print(df.describe(include='object'))
print("\nContagem de valores nulos por coluna:")
print(df.isnull().sum())

### Identificando Features e Target


In [ ]:
X = df.drop('HeartDisease', axis=1)
y = df['HeartDisease']

print("Shape das Features (X):", X.shape)
print("Shape do Target (y):", y.shape)
print("\nPrimeiras 5 linhas das Features (X):")
print(X.head())
print("\nPrimeiras 5 ocorrências do Target (y):")
print(y.head())

## 2. Divisão de Dados


In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print("Shape de X_train:", X_train.shape)
print("Shape de X_val:", X_val.shape)
print("Shape de y_train:", y_train.shape)
print("Shape de y_val:", y_val.shape)

print("\nProporção do target no conjunto de treino:")
print(y_train.value_counts(normalize=True))
print("\nProporção do target no conjunto de validação:")
print(y_val.value_counts(normalize=True))

## 3. Transformação de Dados

In [ ]:
numerical_features = X_train.select_dtypes(include=np.number).columns.tolist()
categorical_features = X_train.select_dtypes(include='object').columns.tolist()

print("Colunas Numéricas:", numerical_features)
print("Colunas Categóricas:", categorical_features)

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore', drop='first'), categorical_features) 
    ],
    remainder='passthrough' 
)

## 4. Modelagem de K-Nearest Neighbors (KNN)


In [ ]:
initial_k = 5
knn_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', KNeighborsClassifier(n_neighbors=initial_k))
])

knn_pipeline.fit(X_train, y_train)

print(f"Modelo KNN treinado com K={initial_k}.")

## 5. Avaliação de Modelos



In [ ]:
y_pred_val = knn_pipeline.predict(X_val)

accuracy = accuracy_score(y_val, y_pred_val)
print(f"Acurácia do modelo KNN (K={initial_k}) no conjunto de validação: {accuracy:.4f}")

## 6. Análise de Parâmetros

In [ ]:
k_values = range(1, 26) 
accuracies = []

print("Analisando diferentes valores de K:")
for k in k_values:
    current_knn_pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('classifier', KNeighborsClassifier(n_neighbors=k))
    ])
    current_knn_pipeline.fit(X_train, y_train)
    
    y_pred_k = current_knn_pipeline.predict(X_val)
    acc_k = accuracy_score(y_val, y_pred_k)
    accuracies.append(acc_k)
    print(f"K = {k:2d}, Acurácia = {acc_k:.4f}")

best_k_index = np.argmax(accuracies)
best_k_value = k_values[best_k_index]
best_accuracy = accuracies[best_k_index]

print(f"\nMelhor valor de K encontrado: {best_k_value}")
print(f"Acurácia correspondente: {best_accuracy:.4f}")

In [ ]:
plt.figure(figsize=(12, 7))
plt.plot(k_values, accuracies, marker='o', linestyle='-', color='b', mfc='red')
plt.title('Acurácia do KNN vs. Valor de K no Conjunto de Validação', fontsize=15)
plt.xlabel('Valor de K (Número de Vizinhos)', fontsize=12)
plt.ylabel('Acurácia', fontsize=12)
plt.xticks(k_values)
plt.grid(True, linestyle='--', alpha=0.7)
plt.axvline(best_k_value, color='green', linestyle=':', label=f'Melhor K = {best_k_value} (Acurácia: {best_accuracy:.4f})')
plt.legend()
plt.show()

## 7. Conclusão

Neste projeto, foi realizada as seguintes etapas:
1.  **Carregamento e Análise Inicial dos Dados**: Foi carregado o dataset `heart.csv`, verificado sua estrutura, tipos de dados e valores ausentes.
2.  **Divisão dos Dados**: Foi separado os dados em conjuntos de treinamento (80%) e validação (20%), utilizando estratificação para manter a proporção das classes do alvo.
3.  **Transformação de Dados**: Foi identificado features numéricas e categóricas. Criado um pipeline de pré-processamento usando `ColumnTransformer` para aplicar `StandardScaler` às features numéricas e `OneHotEncoder` (com `drop='first'`) às features categóricas.
4.  **Modelagem com KNN**: Foi implementado o K-Nearest Neighbors dentro de um `Pipeline` do Scikit-Learn, começando com K=5.
5.  **Avaliação Inicial**: Foi avaliado o modelo inicial no conjunto de validação usando a métrica de acurácia.
6.  **Análise de Parâmetros**: Iterei por uma faixa de valores de K (1 a 25), treinei e avaliei o modelo KNN para cada K, e plotei a acurácia em função de K. Isso permitiu identificar o valor de K que resultou na maior acurácia no conjunto de validação.
